In [ ]:
%matplotlib inline
import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import psycopg2
from dotenv import load_dotenv

In [ ]:
#https://www.youtube.com/watch?v=MEEVcpnfElY
leituras_dique6c = pd.read_csv('../arquivos/leituras_dique6c.csv')
leituras_bvsa = pd.read_csv('../arquivos/leituras_bvsa.csv')

In [ ]:
loc_bvsa = pd.read_csv('../arquivos/loc_bvsa.csv', sep=';', decimal=',')
loc_dique6c = pd.read_csv('../arquivos/loc_dique6c.csv', sep=';', decimal=',')

In [ ]:
loc_sensores = pd.read_excel('../arquivos/LOCALIZAÇÃO_SENSORES.xlsx', engine='openpyxl', header=2)

In [ ]:
leituras_dique6c['area'] = 'BVSA'
leituras_bvsa['area'] = 'Dique 6C'

In [ ]:
leituras = pd.concat([leituras_bvsa, leituras_dique6c])
leituras = leituras.rename(columns={"Sensor":"sensor", "Data":"data", "Valor":"valor", "Unidade":"unidade", "Direcao_Saida":"direcao_saida", "Estado":"estado"})
leituras['direcao_saida'] = leituras['direcao_saida'].replace('Cota NA', 'Cota do NA')

In [ ]:
leituras['sensor'] = leituras['sensor']\
    .apply(lambda \
        x: x.replace("(I)", '')\
            .replace("(S)", '')\
                .replace("‐", '-')\
                    .replace("-ER", '')\
                        .replace("-TC", '')\
                            .strip())

In [ ]:
leituras['direcao_saida'].value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

leituras['id_unidade'] = le.fit_transform(leituras['unidade']) +1
leituras['id_direcao_saida'] = le.fit_transform(leituras['direcao_saida']) +1
leituras['id_estado'] = le.fit_transform(leituras['estado']) +1
leituras['id_area'] = le.fit_transform(leituras['area']) + 1

leituras = leituras.reset_index(drop=True)

In [ ]:
leituras.groupby(by=['unidade', 'direcao_saida', 'estado', 'area']).agg(pd.Series.nunique)[['sensor']].sort_values(by=['sensor'], ascending=False)

In [ ]:
unidade = leituras[['id_unidade', 'unidade']].sort_values(by=['id_unidade']).drop_duplicates()
direcao_saida = leituras[['id_direcao_saida', 'direcao_saida']].sort_values(by=['id_direcao_saida']).drop_duplicates()
estado = leituras[['id_estado', 'estado']].sort_values(by=['id_estado']).drop_duplicates()
area = leituras[['id_area', 'area']].sort_values(by=['id_area']).drop_duplicates()

In [ ]:
#Criação da tabela de sensores

In [ ]:
tabela_sensores = leituras[['sensor']].drop_duplicates().sort_values(by=['sensor']).reset_index(drop=True)
tabela_sensores.index = tabela_sensores.index+1
tabela_sensores.reset_index(drop=False, names='id')

In [ ]:
dict_sensores = tabela_sensores.reset_index(names=['id'])[['sensor','id']].to_dict(orient = 'tight')
dict_sensores = dict(dict_sensores['data']);

In [ ]:
leituras['id_sensor'] = leituras['sensor'].map(dict_sensores)

In [ ]:
leituras['id_sensor'] = leituras['id_sensor'].apply(lambda x: int(x))

In [ ]:
leituras = leituras[['id_sensor','id_unidade', 'id_direcao_saida', 'id_estado', 'id_area', 'sensor', 'data', 'valor', 'unidade', 'direcao_saida', 'estado', 'area']]

In [ ]:
leituras = leituras.drop_duplicates()

In [ ]:
#Tratamento dos dados de localização
import geopandas as gpd

In [ ]:
localizacao = gpd.GeoDataFrame(pd.concat([loc_bvsa, loc_dique6c]))
localizacao = localizacao.rename(columns={"Sensor":"sensor", "Latitude":"latitude", "Longitude":"longitude", "Altura":"altura"})

In [ ]:
localizacao['sensor'] = localizacao['sensor'].apply(lambda x: x.replace("(I)", '').replace("(S)", '').replace("‐", '-').strip())

In [ ]:
localizacao['id_sensor'] = localizacao['sensor'].map(dict_sensores)

In [ ]:
localizacao[localizacao['id_sensor'].isnull()]['sensor'].unique()

In [ ]:
localizacao = localizacao.reset_index(drop=True)
localizacao.index = localizacao.index + 1
localizacao

In [ ]:
x_latitude = localizacao['latitude']
y_longitude = localizacao['longitude']
z_altura = localizacao['altura']

In [ ]:
localizacao['geom'] = gpd.GeoSeries.from_xy(x =y_longitude, y=x_latitude, z=z_altura, crs='EPSG:4326')

In [ ]:
localizacao = localizacao[['id_sensor', 'sensor', 'latitude', 'longitude', 'altura', 'geom']]

In [ ]:
localizacao.head()

In [ ]:
from libary.Database import Database

In [ ]:
database = Database(schema='pesquisa', table='view_leituras_sensores', path='../.env', ambiente = 1)

In [ ]:
localizacao = localizacao.set_geometry('geom')

In [ ]:
localizacao_ = localizacao[['id_sensor', 'sensor', 'latitude', 'longitude', 'altura']]

In [ ]:
leituras = leituras.rename(columns={'index': 'id'})
localizacao = localizacao.rename(columns={'index': 'id'})

In [ ]:
from libary.Database import Database

In [ ]:
tabela_sensores = leituras[['sensor']].drop_duplicates().sort_values(by=['sensor']).reset_index(drop=True)
tabela_sensores.index = tabela_sensores.index+1
tabela_sensores.reset_index(drop=False, names='id');

In [ ]:
database = Database(schema='pesquisa', table='sensores', ambiente = 0, path='../.env')
database.truncate_table()
database.to_table(df=tabela_sensores, schema='pequisa', table='sensores')

In [ ]:
database = Database(schema='pesquisa', table='unidade', ambiente = 0, path='../.env')
database.truncate_table()
database.to_table(df=unidade, schema='pequisa', table='unidade')

In [ ]:
database = Database(schema='pesquisa', table='direcao_saida', ambiente = 0, path='../.env')
database.truncate_table()
database.to_table(df=direcao_saida, schema='pequisa', table='direcao_saida')

In [ ]:
database = Database(schema='pesquisa', table='area', ambiente = 0, path='../.env')
database.truncate_table()
database.to_table(df=area, schema='pequisa', table='area')

In [ ]:
database = Database(schema='pesquisa', table='estado', ambiente = 0, path='../.env')
database.truncate_table()
database.to_table(df=estado, schema='pequisa', table='estado')

In [ ]:
database = Database(schema='pesquisa', table='leituras', ambiente = 0, path='../.env')
database.truncate_table()
database.to_table(df=leituras, schema='pequisa', table='leituras')

In [ ]:
database = Database(schema='pesquisa', table='localizacao', ambiente = 0, path='../.env')
database.truncate_table()
database.to_table(df=localizacao, schema='pequisa', table='localizacao', geom=True)

In [ ]:
tabela_sensores.to_excel('../tabelas/sensores.xlsx')
unidade.to_excel('../tabelas/unidades.xlsx')
direcao_saida.to_excel('../tabelas/direcao_saida.xlsx')
leituras.to_excel('../tabelas/leituras.xlsx')
localizacao.to_excel('../tabelas/localizacao.xlsx')